In [1]:
from pydataset import data
import pandas as pd
import numpy as np

# Load the mpg dataset
mpg = data('mpg')


In [60]:
# On average, which manufacturer has the best miles per gallon?

overall_mpg = mpg.assign(avg_mpg_overall = (mpg.cty + mpg.hwy) / 2)

group_mpg = overall_mpg.groupby('manufacturer').agg({'avg_mpg_overall':'mean','cty':'mean','hwy':'mean'})

group_mpg.rename(columns={'city': 'cty_average', 'hwy': 'hwy_average'}, inplace=True)

# Best city mileage
# group_mpg.sort_values(by = ['city_average'], ascending = False).head(1)

# Best highway mileage
# group_mpg.sort_values(by = ['hwy'], ascending = False).head(1)

# Best overall mileage (city and highway combined)
group_mpg.sort_values(by = ['avg_mpg_overall'], ascending = False).head(1)


,avg_mpg_overall,cty,hwy_average
manufacturer,,,
honda,28.5,24.444444,32.555556


In [3]:
# How many different manufacturers are there?

mpg.manufacturer.agg('nunique')

# alternate answer
# len(mpg.manufacturer.unique())



15

In [4]:
# How many different models are there?

mpg.model.agg('nunique')

# alt answer
# len(mpg.manufacturer.unique())


38

In [52]:
# Do automatic or manual cars have better miles per gallon?

automan = mpg.assign(avg_mpg_overall = (mpg.cty + mpg.hwy) / 2)

automan['trans_cat'] = np.where(mpg.trans.str.startswith('a'), 'auto', 'manual')

automan_comparison = automan.groupby('trans_cat').agg({'avg_mpg_overall':'mean','cty':'mean','hwy':'mean'})

automan_comparison.rename(columns = {'cty': 'city_average', 'hwy': 'hwy_average'}, inplace=True)

# display results 
automan_comparison

# Answer: manual transmissions give the best mpg for city, highway, and overall

,avg_mpg_overall,city_average,hwy_average
trans_cat,,,
auto,19.130573,15.968153,22.292994
manual,22.227273,18.675325,25.779221


In [6]:
from env import host, username, password

url = f'mysql+pymysql://{username}:{password}@{host}/employees'
    
pd.read_sql('SELECT * FROM employees LIMIT 5 OFFSET 50', url)


,emp_no,birth_date,first_name,last_name,gender,hire_date
0,10051,1953-07-28,Hidefumi,Caine,M,1992-10-15
1,10052,1961-02-26,Heping,Nitsch,M,1988-05-21
2,10053,1954-09-13,Sanjiv,Zschoche,F,1986-02-04
3,10054,1957-04-04,Mayumi,Schueller,M,1995-03-13
4,10055,1956-06-06,Georgy,Dredge,M,1992-04-27


In [24]:
avg_mpg_dataframe2 = average_mpg_overall_df.groupby('trans').agg({'avg_mpg_overall':'mean','cty':'mean','hwy':'mean'})

In [25]:
avg_mpg_dataframe2

,avg_mpg_overall,cty,hwy
trans,,,
auto(av),23.900000,20.000000,27.800000
auto(l3),24.000000,21.000000,27.000000
auto(l4),18.951807,15.939759,21.963855
auto(l5),17.717949,14.717949,20.717949
auto(l6),16.833333,13.666667,20.000000
auto(s4),22.166667,18.666667,25.666667
auto(s5),21.333333,17.333333,25.333333
auto(s6),21.281250,17.375000,25.187500
manual(m5),22.775862,19.258621,26.293103
